This notebook demonstrates several geospatial processing tasks, beginning with the conversion of GDB and GML files into the FlatGeobuf (FGB) format for improved handling of spatial data. The processed data is then queried and filtered using DuckDB, which efficiently handles spatial queries on the local machine.

Two primary filtering approaches are applied to the locally stored dataset:

* Direct Filtering:
The first two filtering methods operate directly on the file already saved on the local machine. One method quickly retrieves an initial subset (built areas) to provide a quick overview, while the other selects all rows within a 500-meter radius of a chosen point, capturing entries for all three species.

* Streaming-Based Filtering:
The remaining two methods perform nearly identical filtering as the direct methods but include sleep delays to simulate streaming. This demonstrates how the data can be processed incrementally in batches as it is ingested over time.

The notebook presents the filtered results in both tabular format and interactive map form, offering clear visualizations of the spatial filtering outcomes.

# Import part

In [81]:
import os
import geopandas as gpd
import fiona
import duckdb
import folium
from shapely import wkt
from shapely.geometry import Point, Polygon
import time
import random
import pandas as pd
from IPython.display import display, clear_output

# Functions and classes


In [82]:
# Class that convert bytes to mb and kb
class FileSizeConverter:
    def human_readable_size(self, size):
        # If the size is greater than 100,000 bytes -> convert to megabytes
        if size > 100000:
            size_mb = size / (1024 ** 2)
            return f"{size_mb:.2f} MB"
        else:
            # Otherwise -> convert to kilobytes
            size_kb = size / 1024
            return f"{size_kb:.2f} KB"

In [83]:
# Function that connects to DuckDB
def get_duckdb_connection():
    con = duckdb.connect()
    # Install and load the spatial extension for DuckDB.
    con.execute("INSTALL spatial; LOAD spatial;")
    return con

In [84]:
# Function that export the result to flatGeoBuf.
def export_gdf_with_stats(gdf, output_path, original_file_path):
    # Start the timer to measure export time
    start_time = time.time()

    # Export the GeoDataFrame to a FlatGeoBuf file.
    gdf.to_file(output_path, driver="FlatGeobuf")

    # Stop the timer after export.
    end_time = time.time()
    elapsed_time = end_time - start_time # Calculate the total time

    # Get the file size in bytes
    orig_size_bytes = os.path.getsize(original_file_path)
    exported_size_bytes = os.path.getsize(output_path)

    # Convert file sizes to human-readable format using fileSizeConverter
    converter = FileSizeConverter()
    orig_size_str = converter.human_readable_size(orig_size_bytes)
    exported_size_str = converter.human_readable_size(exported_size_bytes)

    # Print out information about the downloaded file and original file.
    print(f"GeoDataFrame er lagret til: {output_path}")
    print(f"Tid brukt på eksport: {elapsed_time:.2f} sekunder")
    print(f"Original fil ({original_file_path}) størrelse: {orig_size_str}")
    print(f"Eksportert fil størrelse: {exported_size_str}")

In [85]:
# Function that determine abcolor based on the tree species value
def get_color(treslag):
    try:
        value = int(float(treslag))
    except Exception:
        value = None
    color_map = {
        31: "blue",
        32: "green",
        33: "red",
        39: "purple"
    }
    return color_map.get(value, "gray")

# function for GeoJSON layer that uses the treslag property to set feature colors
def style_function(feature):
    treslag = feature['properties'].get('treslag')
    color = get_color(treslag)
    return {
        'fillColor': color,
        'color': color,
        'weight': 1,
        'fillOpacity': 0.5,
    }

# Converting to FGB file format

In [86]:
# Function that converting file til FGB
def convert_to_flatgeobuf(input_file, output_file, layer=None):
    # Get the file extension in lowercase
    file_ext = os.path.splitext(input_file)[1].lower()

    # For GML files, the layer parameter must be specified.
    if file_ext == '.gml':
        if layer is None:
            raise ValueError("For GML-files, 'layer' must be specified (e.g., 'ArealressursFlate').")
        gdf = gpd.read_file(input_file, layer=layer)
    # For GDB-files, the layer can be automatically determined if not specified.
    elif file_ext == '.gdb':
        if layer is None:
            # List all available layers and choose the first one.
            layers = fiona.listlayers(input_file)
            print("Tilgjengelige lag i GDB:", layers)
            layer = layers[0]
        gdf = gpd.read_file(input_file, layer=layer)
    else:
        raise ValueError(f"Ukjent eller ikke støttet filtype: {file_ext}")

    # Filter the geodataframe to retain only polygons (polygon and multipolygon)
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]
    print("Geometrityper etter filtrering:")
    print(gdf.geom_type.value_counts())

    # Export the filtered GeoDataFrame to a FlatGeobuf (FGB) file.
    gdf.to_file(output_file, driver="FlatGeobuf")
    print(f"Lagret polygoner som {output_file}")

    return gdf
# Example usage:
input_gml = "data/42_25832_ar50_gml.gml"
input_gdb = "data/AR50_gdb.gdb"
output_fgb = "data/AR50_flater.fgb"
# For the GDB file, you must specify ar50 as the layer
# For the GML file, you must specify ArealressursFlate as the layer.
convert_to_flatgeobuf(input_gml, output_fgb, layer="ArealressursFlate")

Geometrityper etter filtrering:
Polygon    179790
Name: count, dtype: int64
Lagret polygoner som data/AR50_flater.fgb


,gml_id,lokalId,navnerom,informasjon,områdeId,originalDatavert,kopidato,målemetode,oppdateringsdato,arealtype,skogbonitet,treslag,jordbruk,vegetasjonsdekke,kartstandard,geometry
0,idfbb8c036-19e3-4a9a-b2f6-0c2d35695c1d,176214,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,13,31,98,98,AR50,"POLYGON ((485218.613 6517125.999, 485201.686 6..."
1,id36b5f040-baf2-4727-b0a1-cc90cb4bd608,176213,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,11,99,98,98,AR50,"POLYGON ((485586.131 6519504.745, 485607.835 6..."
2,ide33f3b18-38c2-4514-ae13-ca2df0a2e41a,176212,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,12,31,98,98,AR50,"POLYGON ((487006.634 6519259.077, 486992.389 6..."
3,idd556feed-1ea8-48c4-af32-9ad8e3da9fe7,176211,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,11,39,98,98,AR50,"POLYGON ((483970.068 6519640.093, 483990.43 65..."
4,id5f39e277-26d6-42b0-bacf-f81ac3a628d2,176210,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,11,31,98,98,AR50,"POLYGON ((485413.573 6520531.763, 485458.321 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179785,id8d863682-6cb3-4c6b-9190-8eefa6cf6633,5,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,11,39,98,98,AR50,"POLYGON ((358617.147 6442580.922, 358631.381 6..."
179786,idff57878f-0fbb-4ee5-9fa5-d8e364e9ff58,4,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,13,31,98,98,AR50,"POLYGON ((358002.134 6441284.427, 357998.139 6..."
179787,idbbd694b0-068f-4308-ab6e-8c5d3f34e011,3,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,50,98,39,98,55,AR50,"POLYGON ((357579.6 6444891.118, 357578.268 644..."
179788,idce0ff6b8-8fa6-4cb7-8476-08e2415bb4ad,2,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,20,98,98,24,98,AR50,"POLYGON ((357225.405 6444709.267, 357232.189 6..."


# Filtering

## Top 10 developed areas in Kristiansand

The filtering retrieves data from a dataset by selecting rows where "områdeid" equals 4204 (representing Kristiansand) and "arealtype" equals 10 (indicating built-up areas). It then limits the output to the first 10 rows, providing a focused snapshot of the data for that specific region and type.

In [87]:
# Obtain a connection to DuckDB, which will be used for spatial queries
con = get_duckdb_connection()


# Reads from fgb file, then select columns "gml_id", "områdeId", "arealtype".
# Convert geometry to its Well-Known text (WKT) and filter rows where arealtype = 10 and områdeid = 4204
# Limit the result to the first 10 rows
query = f"""
    SELECT
        "gml_id",
        "områdeId",
        "arealtype",
        ST_AsText("geom") AS geometry_wkt
    FROM ST_Read('data/AR50_flater.fgb')
    WHERE "arealtype" = 10 AND "områdeID" = 4204
    LIMIT 10
"""
# Execute the query and fetch the result as a pandas DataFrame, then display it
result = con.execute(query).fetchdf()
display(result)

,gml_id,områdeId,arealtype,geometry_wkt
0,id2a697c97-a491-493c-bd59-d1fb17d2ffd9,4204,10,POLYGON ((450127.17372007103 6441551.161033999...
1,id6eff95a9-ef0d-4747-bad0-5abba81a0965,4204,10,"POLYGON ((449033.6828441841 6441453.525762451,..."
2,id365ec336-6e15-44db-b548-66d34fc221a1,4204,10,"POLYGON ((449722.259151386 6442046.072154977, ..."
3,idbe38f0c8-c57b-465d-a371-2e2684e39486,4204,10,"POLYGON ((448986.6394210853 6441980.386953776,..."
4,id913614cc-473a-43de-862e-b7b585a4f652,4204,10,"POLYGON ((448936.5817574248 6441755.476563986,..."
5,idf4070f92-1e8c-4b1c-8a8d-89fcd52a75a9,4204,10,"POLYGON ((440547.0395784581 6438311.181694474,..."
6,id906ca7db-5606-4aae-a46c-4fd1c43f54c3,4204,10,"POLYGON ((439695.3894747687 6437974.65795368, ..."
7,id449f63fd-2346-4a2c-aa0c-cbd7075e6eae,4204,10,"POLYGON ((439665.3768998971 6437300.694913746,..."
8,id136ed6bd-761c-47ff-aa33-0c1e0d026993,4204,10,POLYGON ((439640.15813446033 6436797.486323194...
9,id1976a7dc-07df-4f28-a54b-11378a44e61e,4204,10,"POLYGON ((436810.9030575423 6439310.181405061,..."


In [88]:
# Convert the WKT string to a shapely geometry object
result['geometry'] = result['geometry_wkt'].apply(wkt.loads)

# Create a GeoDataFrame from the DataFrame using the geometry column
# Set the coordinate reference system (CRS) to EPSG:25832
gdf = gpd.GeoDataFrame(result, geometry='geometry')
gdf.set_crs("EPSG:25832", inplace=True)

# Convert the områdeid column to an integer and the nto a string
# This step is done to remove thousand separators and ensure consistent formatting
gdf["områdeId"] = gdf["områdeId"].astype(int).astype(str)

# Create a new file with only the filtered result
output_path_1 = "data/filter1_direct.fgb"
original_file_path = "data/AR50_flater.fgb"
export_gdf_with_stats(gdf, output_path_1, original_file_path)

# Calculate the centroid of the geometries
gdf_projected = gdf.to_crs("EPSG:3857")
center_point_projected = gdf_projected.geometry.union_all().centroid

# Transform the centroid from EPSG:3857 back to EPSG:4326 (latitude/longitude) for displaying on a Folium map.
center_point = gpd.GeoSeries([center_point_projected], crs="EPSG:3857").to_crs("EPSG:4326").iloc[0]
center_lat, center_lon = center_point.y, center_point.x

# Reproject the GeoDataFrame to EPSG:4326 for display purposes with Folium.
gdf = gdf.to_crs("EPSG:4326")

# Create a folium map centered at the calculated centroid with an initial zooom level of 12
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Convert GeoDataFrame to GeoJSON and define a popup configuration
popup_fields = ["arealtype", "områdeId"]
popup = folium.GeoJsonPopup(
    fields=popup_fields,
    aliases=[f"{col}:" for col in popup_fields],
    localize=True,
    labels=True,
    style="background-color: white;",
)

# Convert the GeoDataFrame to a GeoJSON string
geojson_data = gdf.to_json()

# Add a geoJSON layer to the folium map with style
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.5
    },
    popup=popup
).add_to(m)

# Display the map
m


GeoDataFrame er lagret til: data/filter1_direct.fgb
Tid brukt på eksport: 0.00 sekunder
Original fil (data/AR50_flater.fgb) størrelse: 288.94 MB
Eksportert fil størrelse: 0.10 MB


## Tree species identification by coordinates: A radius search

This filter uses a specific point defined by its coordinates as the center, and then creates a 500-meter buffer around that point. It examines the dataset within this area to determine which tree species (treslag) are present.

### Cleaning the data

In this section, the data cleaning process addresses non-informative values in the dataset. Specifically, numerical codes such as 98 and 99, which indicate "not registered", are converted to null values. This step is crucial because it ensures that these placeholder values are ignored during subsequent queries. Without this conversion, queries such as aggregating or filtering based on tree species in a given area could yield incorrect or misleading results. By setting these values to null, the filtering operations can accurately reflect the real data, providing reliable outcomes for analyses.

In [89]:
# Connection to DuckDB
con = get_duckdb_connection()

con.execute("""
CREATE OR REPLACE TEMPORARY VIEW renset_data AS
SELECT
  CASE WHEN arealtype IN (98, 99) THEN NULL ELSE arealtype END AS arealtype,
  CASE WHEN skogbonitet IN (98, 99) THEN NULL ELSE skogbonitet END AS skogbonitet,
  CASE WHEN treslag IN (98, 99) THEN NULL ELSE treslag END AS treslag,
  CASE WHEN jordbruk IN (98, 99) THEN NULL ELSE jordbruk END AS jordbruk,
  CASE WHEN vegetasjonsdekke IN (98, 99) THEN NULL ELSE vegetasjonsdekke END AS vegetasjonsdekke,
  *
FROM ST_READ('data/AR50_flater.fgb')
""")

### Coordinates and buffer

In [90]:
# Define the center point with latitude and longitude
center_lat = 58.1743128719827
center_lon = 8.01122450699552
center = Point(center_lon, center_lat)
center_gs = gpd.GeoSeries([center], crs="EPSG:4326")

# Reproject the center point to the datasets CRS (EPSG:25832)
center_25832 = center_gs.to_crs("EPSG:25832").iloc[0]
buffer_25832 = center_25832.buffer(500) # 500m buffer around the center
buffer_wkt = buffer_25832.wkt


### Query

In [91]:
# Query fetch the data within the buffer
query = f"""
SELECT
  gml_Id,
  lokalId,
  treslag,
  områdeid,
  ST_AsText(ST_Intersection("geom", ST_GeomFromText('{buffer_wkt}', false))) AS geometry_wkt
FROM renset_data
WHERE treslag IS NOT NULL
  AND ST_Intersects("geom", ST_GeomFromText('{buffer_wkt}', false));
"""

# Execute the query and fetch the result as a DataFrame, then display it
result = con.execute(query).fetchdf()
display(result)

,gml_id,lokalId,treslag,områdeId,geometry_wkt
0,idabd9cde9-8983-400f-b1f0-a24e8bd1a1e9,58737,33,4204,MULTIPOLYGON (((442257.6728512464 6448817.9104...
1,id1bc5f7f7-8d7c-4ba4-b74d-545ac36e92c9,51100,33,4204,"POLYGON ((442170.9196794048 6448595.852558057,..."
2,ida98b9802-c7fb-4e66-a5cc-541bcfed4b05,49275,31,4204,"POLYGON ((442172.7287056746 6448743.802758148,..."
3,idd64d9bb8-75d4-47f1-9ddb-46a14c1d107c,58590,31,4204,"POLYGON ((442183.5995492529 6448762.308386278,..."
4,id3ef6a438-d33b-4e38-a29c-1e9fbe0393f9,51116,31,4204,MULTIPOLYGON (((441646.9019362646 6448321.0844...
5,id01944ea4-fc73-45a9-a8cc-d94b10b3b375,51114,31,4204,POLYGON ((441936.33804094495 6448081.429082607...
6,ide5029f39-9558-478d-8928-7be27998d11e,51113,33,4204,"POLYGON ((442065.0455018429 6448491.206325524,..."
7,idd1e215e9-f900-490a-99ba-5e6194b87093,58548,31,4204,POLYGON ((442074.93349893455 6448604.809835015...
8,id33a1874a-c9d6-46cf-a5d1-5ee5760cf9ca,53990,31,4204,"POLYGON ((442167.9055979869 6448545.485427696,..."
9,id47c56cba-5ed8-4354-a602-d21dc33c8479,58660,33,4204,"POLYGON ((442166.6581993208 6448615.510891866,..."


### Coordinate Visualization

In [92]:
# Convert WKT strings into shapely geometry object - creates a Geodataframe
result['geometry'] = result['geometry_wkt'].apply(shapely.wkt.loads)
gdf = gpd.GeoDataFrame(result, geometry='geometry')
gdf = gdf.set_crs("EPSG:25832", allow_override=True)

# Export the filtered GeoDataFrame to a FGB file
output_path_2 = "data/filter2_direct.fgb"
original_file_path = "data/AR50_flater.fgb"
export_gdf_with_stats(gdf, output_path_2, original_file_path)

# Reprosject the geodataframe to a EPSG:4326 for visualization with folium
gdf = gdf.to_crs("EPSG:4326")

# Create a folium map centered to the defined center point
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Popup for the geoJSON layer to display the tree species values when a feature is clicked
popup = folium.GeoJsonPopup(
    fields=['treslag'],
    aliases=['Treslag: '],
    localize=True,
    labels=True,
    style="background-color: white;"
)

# Add geoJSON layer to the folium map with style and popup
folium.GeoJson(
    data=gdf.to_json(),
    style_function=style_function,
    popup=popup
).add_to(m)

# Display on map
m


GeoDataFrame er lagret til: data/filter2_direct.fgb
Tid brukt på eksport: 0.00 sekunder
Original fil (data/AR50_flater.fgb) størrelse: 288.94 MB
Eksportert fil størrelse: 55.45 KB


# Testing using sleep to mimic network transfer delays

### Sleep function

In [93]:
# Simulates streaming of geospatial data by splitting the full dataset into smaller batches with an artificial delay
def stream_geo_data(query, batch_size=5):
    # Execute the query and fetch all results into a DataFrame
    all_results = con.execute(query).fetchdf()
    total_rows = len(all_results)

    # Split the result into batches of size "batch_size"
    for i in range(0, total_rows, batch_size):
        batch = all_results.iloc[i:i+batch_size].copy()

        # Simulate a network delay by sleeping for random interval between o.5 and 1.5 seconds
        delay = random.uniform(0.5, 1.5)
        time.sleep(delay)

        # Print a message showing wich batch has been fetched and how many rows it contains
        print(f"Hentet batch {i//batch_size + 1} med {len(batch)} rader. Neste batch om {delay:.2f} sekunder...")
        yield batch

## Filtering: Top 10 developed areas in Kristiansand

### Query

In [94]:
# Define a query to read 10 rows from fgb file
query = """
    SELECT
        "gml_id",
        "områdeId",
        "arealtype",
        ST_AsText("geom") AS geometry_wkt
    FROM ST_Read('data/AR50_flater.fgb')
    WHERE "arealtype" = 10 AND "områdeID" = 4204
    LIMIT 10
"""

### Print result in table format and map

In [95]:
# Display the initial message and prepare to collect streamed results
print("\nHenter de 10 første bebygde områdene:")
all_results = []

# Loop over streamed batches (using a batch size of 2 rows)
for batch in stream_geo_data(query, batch_size=2):
    # Convert the WKT strings in the geometry_wkt column to shapely geometry objects
    batch['geometry'] = batch['geometry_wkt'].apply(wkt.loads)
    all_results.append(batch)

# Combine all batches into one DataFrame
final_result = pd.concat(all_results, ignore_index=True)

# Display the complete table of 10 rows
print("\nViser alle 10 rader i tabellform:")
display(final_result)

# Create a GeoDataFrame from the final dataframe for mapping
gdf = gpd.GeoDataFrame(final_result, geometry='geometry')
gdf.set_crs("EPSG:25832", inplace=True)

# Convert the områdeid column to an integer and then to a string to remove thousand separators
gdf["områdeId"] = gdf["områdeId"].astype(int).astype(str)

# Export the GeoDataframe to a FGB fil with ecport statistics
output_path_3 = "data/filter1_stream.fgb"
original_file_path = "data/AR50_flater.fgb"
export_gdf_with_stats(gdf, output_path_3, original_file_path)


# Reproject the GeodataFrame to EPSG:3857 for accurate centroid calculation
gdf_projected = gdf.to_crs("EPSG:3857")
center_point_projected = gdf_projected.geometry.union_all().centroid

# Transform the centroid back to EPSG:4326 lat/long for folium
center_point = gpd.GeoSeries([center_point_projected], crs="EPSG:3857").to_crs("EPSG:4326").iloc[0]
center_lat, center_lon = center_point.y, center_point.x

# Reproject the GeoDataFrame to EPSG:4326
gdf = gdf.to_crs("EPSG:4326")

# Create a Folium map centered at the calculated centroid with an initial zoom level of 12
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Define popup fields and settings to show only 'arealtype' and 'områdeId'.
popup_fields = ["arealtype", "områdeId"]
popup = folium.GeoJsonPopup(
    fields=popup_fields,
    aliases=[f"{col}:" for col in popup_fields],
    localize=True,
    labels=True,
    style="background-color: white;",
)

# Convert the GeoDataFrame to a GeoJSON string
geojson_data = gdf.to_json()

# Add the GeoJSON data as a layer on the folium map with a predefined style
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.5
    },
    popup=popup
).add_to(m)

# Display map
display(m)


Henter de 10 første bebygde områdene:
Hentet batch 1 med 2 rader. Neste batch om 0.70 sekunder...
Hentet batch 2 med 2 rader. Neste batch om 0.68 sekunder...
Hentet batch 3 med 2 rader. Neste batch om 1.12 sekunder...
Hentet batch 4 med 2 rader. Neste batch om 1.33 sekunder...
Hentet batch 5 med 2 rader. Neste batch om 0.93 sekunder...

Viser alle 10 rader i tabellform:


,gml_id,områdeId,arealtype,geometry_wkt,geometry
0,id2a697c97-a491-493c-bd59-d1fb17d2ffd9,4204,10,POLYGON ((450127.17372007103 6441551.161033999...,POLYGON ((450127.17372007103 6441551.161033999...
1,id6eff95a9-ef0d-4747-bad0-5abba81a0965,4204,10,"POLYGON ((449033.6828441841 6441453.525762451,...","POLYGON ((449033.6828441841 6441453.525762451,..."
2,id365ec336-6e15-44db-b548-66d34fc221a1,4204,10,"POLYGON ((449722.259151386 6442046.072154977, ...","POLYGON ((449722.259151386 6442046.072154977, ..."
3,idbe38f0c8-c57b-465d-a371-2e2684e39486,4204,10,"POLYGON ((448986.6394210853 6441980.386953776,...","POLYGON ((448986.6394210853 6441980.386953776,..."
4,id913614cc-473a-43de-862e-b7b585a4f652,4204,10,"POLYGON ((448936.5817574248 6441755.476563986,...","POLYGON ((448936.5817574248 6441755.476563986,..."
5,idf4070f92-1e8c-4b1c-8a8d-89fcd52a75a9,4204,10,"POLYGON ((440547.0395784581 6438311.181694474,...","POLYGON ((440547.0395784581 6438311.181694474,..."
6,id906ca7db-5606-4aae-a46c-4fd1c43f54c3,4204,10,"POLYGON ((439695.3894747687 6437974.65795368, ...","POLYGON ((439695.3894747687 6437974.65795368, ..."
7,id449f63fd-2346-4a2c-aa0c-cbd7075e6eae,4204,10,"POLYGON ((439665.3768998971 6437300.694913746,...","POLYGON ((439665.3768998971 6437300.694913746,..."
8,id136ed6bd-761c-47ff-aa33-0c1e0d026993,4204,10,POLYGON ((439640.15813446033 6436797.486323194...,POLYGON ((439640.15813446033 6436797.486323194...
9,id1976a7dc-07df-4f28-a54b-11378a44e61e,4204,10,"POLYGON ((436810.9030575423 6439310.181405061,...","POLYGON ((436810.9030575423 6439310.181405061,..."


GeoDataFrame er lagret til: data/filter1_stream.fgb
Tid brukt på eksport: 0.01 sekunder
Original fil (data/AR50_flater.fgb) størrelse: 288.94 MB
Eksportert fil størrelse: 0.10 MB


## Filtering: Tree species identification by coordinates: A radius search

### Cleaning the data.

In [96]:
# Opprett tilkobling
con = get_duckdb_connection()

con.execute("""
CREATE OR REPLACE TEMPORARY VIEW renset_data AS
SELECT
  CASE WHEN arealtype IN (98, 99) THEN NULL ELSE arealtype END AS arealtype,
  CASE WHEN skogbonitet IN (98, 99) THEN NULL ELSE skogbonitet END AS skogbonitet,
  CASE WHEN treslag IN (98, 99) THEN NULL ELSE treslag END AS treslag,
  CASE WHEN jordbruk IN (98, 99) THEN NULL ELSE jordbruk END AS jordbruk,
  CASE WHEN vegetasjonsdekke IN (98, 99) THEN NULL ELSE vegetasjonsdekke END AS vegetasjonsdekke,
  *
FROM ST_READ('data/AR50_flater.fgb')
""")

### Location and buffer

In [97]:
# Simulate a delay to mimic processing time bfore creating the view
time.sleep(random.uniform(1.0, 2.0))

# Define the center point and calculate the buffer
time.sleep(random.uniform(0.5, 1.0))

# Center point using latitude and longitude
center_lat = 58.1743128719827
center_lon = 8.01122450699552
center = Point(center_lon, center_lat)
center_gs = gpd.GeoSeries([center], crs="EPSG:4326")

# Reproject the center to the datasets CRS (EPSG:25832) and create a 500m buffer
center_25832 = center_gs.to_crs("EPSG:25832").iloc[0]
buffer_25832 = center_25832.buffer(500)
buffer_wkt = buffer_25832.wkt

Temporær visning opprettet.

Beregner geografisk buffer...


### Query

In [98]:
# Construct the SQL query to fetch data within the buffer
print("\nHenter data innenfor buffer...")

query = f"""
SELECT
  gml_id,
  lokalId,
  treslag,
  områdeId,
  ST_AsText(ST_Intersection("geom", ST_GeomFromText('{buffer_wkt}', false))) AS geometry_wkt
FROM renset_data
WHERE treslag IS NOT NULL
  AND ST_Intersects("geom", ST_GeomFromText('{buffer_wkt}', false));
"""


Henter data innenfor buffer...


### Vizualize in table and map

In [99]:
# Retrieve and process the data in batches using a streaming simulation.
batches = []
for batch in stream_geo_data(query, batch_size=3):
    # Convert WKT string into shapely geometry objects for each batch
    batch['geometry'] = batch['geometry_wkt'].apply(shapely.wkt.loads)
    batches.append(batch)

# Combine all the batches into a single DataFrame
result = pd.concat(batches, ignore_index=True)
print(f"\nTotalt hentet {len(result)} rader innenfor buffer.")

# Display the result as a table
print("\nViser data innenfor buffer:")
display(result)

# Concert the Datafrme to a geodataframe and assign the CRS
time.sleep(random.uniform(0.5, 1.0))

gdf = gpd.GeoDataFrame(result, geometry='geometry')
gdf = gdf.set_crs("EPSG:25832", allow_override=True)

# Export the processed GeoDataFrame to a FlatGeobuf file along with export statistics.
output_path_4 = "data/filter2_stream.fgb"
original_file_path = "data/AR50_flater.fgb"
export_gdf_with_stats(gdf, output_path_4, original_file_path)

# Reproject the GeoDataFrame to EPSG:4326 for visualization in Folium.
gdf = gdf.to_crs("EPSG:4326")

# Generate a folium map centered a tthe specified point
time.sleep(random.uniform(0.5, 1.0))

m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Popup for the GeoJSON layer to display the tree species value
popup = folium.GeoJsonPopup(
    fields=['treslag'],
    aliases=['Treslag: '],
    localize=True,
    labels=True,
    style="background-color: white;"
)

# Add the GeoJSON layer to the map with defiend styling and popup configuration
folium.GeoJson(
    data=gdf.to_json(),
    style_function=style_function,
    popup=popup
).add_to(m)

print("\nKartet er klart:")
# Display map
display(m)


Hentet batch 1 med 3 rader. Neste batch om 0.61 sekunder...
Hentet batch 2 med 3 rader. Neste batch om 1.50 sekunder...
Hentet batch 3 med 3 rader. Neste batch om 0.74 sekunder...
Hentet batch 4 med 3 rader. Neste batch om 0.51 sekunder...
Hentet batch 5 med 3 rader. Neste batch om 0.64 sekunder...
Hentet batch 6 med 1 rader. Neste batch om 1.38 sekunder...

Totalt hentet 16 rader innenfor buffer.

Viser data innenfor buffer:


,gml_id,lokalId,treslag,områdeId,geometry_wkt,geometry
0,idabd9cde9-8983-400f-b1f0-a24e8bd1a1e9,58737,33,4204,MULTIPOLYGON (((442257.6728512464 6448817.9104...,MULTIPOLYGON (((442257.6728512464 6448817.9104...
1,id1bc5f7f7-8d7c-4ba4-b74d-545ac36e92c9,51100,33,4204,"POLYGON ((442170.9196794048 6448595.852558057,...","POLYGON ((442170.9196794048 6448595.852558057,..."
2,ida98b9802-c7fb-4e66-a5cc-541bcfed4b05,49275,31,4204,"POLYGON ((442172.7287056746 6448743.802758148,...","POLYGON ((442172.7287056746 6448743.802758148,..."
3,idd64d9bb8-75d4-47f1-9ddb-46a14c1d107c,58590,31,4204,"POLYGON ((442183.5995492529 6448762.308386278,...","POLYGON ((442183.5995492529 6448762.308386278,..."
4,id3ef6a438-d33b-4e38-a29c-1e9fbe0393f9,51116,31,4204,MULTIPOLYGON (((441646.9019362646 6448321.0844...,MULTIPOLYGON (((441646.9019362646 6448321.0844...
5,id01944ea4-fc73-45a9-a8cc-d94b10b3b375,51114,31,4204,POLYGON ((441936.33804094495 6448081.429082607...,POLYGON ((441936.33804094495 6448081.429082607...
6,ide5029f39-9558-478d-8928-7be27998d11e,51113,33,4204,"POLYGON ((442065.0455018429 6448491.206325524,...","POLYGON ((442065.0455018429 6448491.206325524,..."
7,idd1e215e9-f900-490a-99ba-5e6194b87093,58548,31,4204,POLYGON ((442074.93349893455 6448604.809835015...,POLYGON ((442074.93349893455 6448604.809835015...
8,id33a1874a-c9d6-46cf-a5d1-5ee5760cf9ca,53990,31,4204,"POLYGON ((442167.9055979869 6448545.485427696,...","POLYGON ((442167.9055979869 6448545.485427696,..."
9,id47c56cba-5ed8-4354-a602-d21dc33c8479,58660,33,4204,"POLYGON ((442166.6581993208 6448615.510891866,...","POLYGON ((442166.6581993208 6448615.510891866,..."



Prosesserer geografiske data...
GeoDataFrame er lagret til: data/filter2_stream.fgb
Tid brukt på eksport: 0.01 sekunder
Original fil (data/AR50_flater.fgb) størrelse: 288.94 MB
Eksportert fil størrelse: 55.45 KB

Genererer kart...

Kartet er klart:
